In [1]:
pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
import pandas as pd
import great_expectations as ge

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test-app").enableHiveSupport().getOrCreate()

24/03/14 18:16:56 WARN Utils: Your hostname, spark-central resolves to a loopback address: 127.0.1.1; using 10.0.10.3 instead (on interface ens160)
24/03/14 18:16:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/14 18:16:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 18:16:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark

In [7]:
spark.sql("show tables").show(100)

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
| default|  job_postings|      false|
| default|    job_skills|      false|
| default|   job_summary|      false|
| default|      mvbarten|      false|
| default|        nested|      false|
| default|         pokes|      false|
| default|       pokes16|      false|
| default|        pokes2|      false|
| default|          tmp1|      false|
| default|          tmp2|      false|
| default|        user12|      false|
| default|user12_parquet|      false|
| default|        user16|      false|
| default|   user21table|      false|
| default|        user22|      false|
| default|        user24|      false|
| default|        user31|      false|
| default|user31_parquet|      false|
| default|        user35|      false|
| default|        user41|      false|
| default|        user45|      false|
| default|user45_parquet|      false|
| default|         user6|      false|
+--------+--

In [18]:
sdf = spark.table("mvbarten").select("id", "budget")

In [19]:
pdf = sdf.toPandas()

24/03/14 18:20:07 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.


In [20]:
pdf

,id,budget
0,NaN,NaN
1,0.0,237000000.0
2,1.0,300000000.0
3,2.0,245000000.0
4,3.0,250000000.0
...,...,...
5797,5793.0,0.0
5798,5794.0,0.0
5799,5795.0,0.0
5800,5796.0,0.0


In [9]:
#Create a pandas dataframe
pd_df = pd.DataFrame.from_dict(data)

In [10]:
pd_df

,Category,ID,Price,Status
0,A,101,55.00,True
1,B,102,200.50,False
2,C,103,9.99,None
3,E,104,18.35,True


In [11]:
type(pd_df)

pandas.core.frame.DataFrame

In [12]:
import pyspark.pandas as spd
sdf = spd.from_pandas(pd_df)

ModuleNotFoundError: No module named 'pyspark.pandas'

In [14]:
#Use pandas to avoid needing to define schema
sdf = spark.createDataFrame(
  pd_df
)

AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [11]:
sdf

NameError: name 'sdf' is not defined

In [10]:
sdf.show()

+--------+---+-----+------+
|Category| ID|Price|Status|
+--------+---+-----+------+
|       A|101| 55.0|  true|
|       B|102|200.5| false|
|       C|103| 9.99|  null|
|       E|104|18.35|  true|
+--------+---+-----+------+



In [11]:
type(sdf)

pyspark.sql.dataframe.DataFrame

#  great-expectations objects


In [21]:
# with pandas we create a great expectations object like this
pd_df_ge = ge.from_pandas(pdf)

In [23]:
pd_df_ge.expect_column_max_to_be_between("id", 1, 100)

{
  "success": false,
  "result": {
    "observed_value": 5797.0,
    "element_count": 5802,
    "missing_count": 4,
    "missing_percent": 0.06894174422612892
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Great expectation object from spark dataframe

In [14]:
# With yspark we can do it like this
sdf_ge = ge.dataset.SparkDFDataset(sdf)

In [15]:
sdf_ge

In [ ]:
#sdf_ge.expect

In [16]:
sdf_ge.expect_column_values_to_not_be_null('Status')

{
  "result": {
    "element_count": 4,
    "unexpected_count": 1,
    "unexpected_percent": 25.0,
    "unexpected_percent_total": 25.0,
    "partial_unexpected_list": [
      null
    ]
  },
  "success": false,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [19]:
sdf_ge.expect_column_values_to_be_between('Price', 1, 100)

{
  "result": {
    "element_count": 4,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 25.0,
    "unexpected_percent_total": 25.0,
    "unexpected_percent_nonmissing": 25.0,
    "partial_unexpected_list": [
      200.5
    ]
  },
  "success": false,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [20]:
sdf_ge.expect_column_to_exist(['Price'])

{
  "result": {},
  "success": false,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [21]:
sdf_ge.expect_column_to_exist('Price')

{
  "result": {},
  "success": true,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Managing GE Spark DataFrame and SparkDataFrame

In [22]:
sdf.count()

4

In [23]:
sdf_ge.count()

AttributeError: 'SparkDFDataset' object has no attribute 'count'

In [24]:
sdf_ge.spark_df.count()

4